## Kamerakalibrierung mit OpenCV
### Mit Hilfe eines Chessboard Musters
- Bilder einlesen aus verzeichnis calib
- Menge an Bildpunkten(imgpoints) und Menge an Weltpunkten(objpoints) initialisieren
- Für jedes Einzelbild Chessboard detektieren
- Bildpunkte & Weltpunkte speichern 
- (optional) erkannte Punkte anzeigen
- Gleichungssystem lösen und Verzeichnungsparameter berechnen
- Aktulalisierte Kameramatrix auf ein Bild anwenden und entzerren

In [1]:
import numpy as np
import cv2 as cv
import glob
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('calib/*.jpg')
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (9,6), None)
    # If found, add object points, image points (after refining them)
    if ret:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (9,6), corners2, ret)
        #cv.imshow('img', img)
        #cv.waitKey(200)
cv.destroyAllWindows()

ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
print(ret, mtx, dist)
print("--------------")

img = cv.imread('calib/left12.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
print(newcameramtx)
# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
cv.imwrite('undistort.png', dst)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult.png', dst)

0.4539405215706796 [[539.54466618   0.         334.65142382]
 [  0.         539.38145314 240.87797289]
 [  0.           0.           1.        ]] [[-0.28893663  0.11328943  0.0006099  -0.00074187 -0.03101715]]
--------------
[[443.63360854   0.         338.27548383]
 [  0.         439.34456121 241.75064326]
 [  0.           0.           1.        ]]


True